In [1]:
# lstm for time series forecasting
from numpy import sqrt
from numpy import asarray
from pandas import read_csv
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM

# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return asarray(X), asarray(y)

# load the dataset
path = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/monthly-car-sales.csv'
df = read_csv(path, header=0, index_col=0, squeeze=True)
# retrieve the values
values = df.values.astype('float32')
# specify the window size
n_steps = 5
# split into samples
X, y = split_sequence(values, n_steps)
# reshape into [samples, timesteps, features]
X = X.reshape((X.shape[0], X.shape[1], 1))
# split into train/test
n_test = 12
X_train, X_test, y_train, y_test = X[:-n_test], X[-n_test:], y[:-n_test], y[-n_test:]
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)
# define model
model = Sequential()
model.add(LSTM(100, activation='relu', kernel_initializer='he_normal', input_shape=(n_steps,1)))
model.add(Dense(50, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(50, activation='relu', kernel_initializer='he_normal'))
model.add(Dense(1))
# compile the model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])
# fit the model
model.fit(X_train, y_train, epochs=350, batch_size=32, verbose=2, validation_data=(X_test, y_test))
# evaluate the model
mse, mae = model.evaluate(X_test, y_test, verbose=0)
print('MSE: %.3f, RMSE: %.3f, MAE: %.3f' % (mse, sqrt(mse), mae))
# make a prediction
row = asarray([18024.0, 16722.0, 14385.0, 21342.0, 17180.0]).reshape((1, n_steps, 1))
yhat = model.predict(row)
print('Predicted: %.3f' % (yhat))

(91, 5, 1) (12, 5, 1) (91,) (12,)
Epoch 1/350
3/3 - 2s - loss: 48936780.0000 - mae: 5946.9258 - val_loss: 41958700.0000 - val_mae: 5208.4741
Epoch 2/350
3/3 - 0s - loss: 38335008.0000 - mae: 5070.9043 - val_loss: 48611680.0000 - val_mae: 5528.2710
Epoch 3/350
3/3 - 0s - loss: 36524156.0000 - mae: 4871.9404 - val_loss: 32641386.0000 - val_mae: 4440.4395
Epoch 4/350
3/3 - 0s - loss: 29343426.0000 - mae: 4111.5107 - val_loss: 26252582.0000 - val_mae: 3874.6858
Epoch 5/350
3/3 - 0s - loss: 20952880.0000 - mae: 3561.1067 - val_loss: 24259286.0000 - val_mae: 4119.2207
Epoch 6/350
3/3 - 0s - loss: 19188440.0000 - mae: 3488.3916 - val_loss: 18430630.0000 - val_mae: 3549.7888
Epoch 7/350
3/3 - 0s - loss: 15935181.0000 - mae: 3228.4612 - val_loss: 16634039.0000 - val_mae: 3347.7961
Epoch 8/350
3/3 - 0s - loss: 14296257.0000 - mae: 3000.6453 - val_loss: 26443886.0000 - val_mae: 4378.9448
Epoch 9/350
3/3 - 0s - loss: 13678143.0000 - mae: 2927.1968 - val_loss: 21363950.0000 - val_mae: 3856.9558
Epo